In [6]:
!pip install faker sqlalchemy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import time
import threading

# Configuração do Faker para o idioma português do Brasil
fake = Faker('pt_BR')
fake_unique_cpf = Faker('pt_BR')

# DataFrames globais para armazenar os dados
clientes_df = pd.DataFrame()
transacoes_df = pd.DataFrame()
emprestimos_df = pd.DataFrame()
pagamentos_df = pd.DataFrame()

# Conjuntos para rastrear IDs únicos
clientes_ids = set()
transacoes_ids = set()
emprestimos_ids = set()
pagamentos_ids = set()

# --- Função para gerar um novo cliente ---
def generate_new_cliente():
    cliente_id = fake.unique.random_number(digits=8)
    while cliente_id in clientes_ids:
        cliente_id = fake.unique.random_number(digits=8)
    clientes_ids.add(cliente_id)
    nome = fake.name()
    cpf = fake_unique_cpf.unique.cpf()
    data_nascimento = fake.date_of_birth(minimum_age=18, maximum_age=70)
    sexo = random.choice(['Masculino', 'Feminino'])
    endereco = fake.address()
    email = fake.email()
    telefone = fake.phone_number()
    return pd.DataFrame([{
        'cliente_id': cliente_id,
        'nome': nome,
        'cpf': cpf,
        'data_nascimento': data_nascimento,
        'sexo': sexo,
        'endereco': endereco,
        'email': email,
        'telefone': telefone
    }])

# --- Função para gerar uma nova transação ---
def generate_new_transacao():
    if clientes_df.empty:
        return None
    transacao_id = fake.unique.random_number(digits=10)
    while transacao_id in transacoes_ids:
        transacao_id = fake.unique.random_number(digits=10)
    transacoes_ids.add(transacao_id)
    cliente_id = random.choice(clientes_df['cliente_id'].tolist())
    data_transacao = fake.date_time_between(start_date='-1y', end_date='now')
    tipo_transacao = random.choice(['Compra', 'Pagamento', 'Depósito', 'Retirada', 'Transferência'])
    valor = round(random.uniform(10, 1000), 2)
    descricao = fake.sentence(nb_words=5)
    return pd.DataFrame([{
        'transacao_id': transacao_id,
        'cliente_id': cliente_id,
        'data_transacao': data_transacao,
        'tipo_transacao': tipo_transacao,
        'valor': valor,
        'descricao': descricao
    }])

# --- Função para gerar um novo empréstimo para um cliente específico ---
def generate_new_emprestimo(cliente_id):
    emprestimo_id = fake.unique.random_number(digits=9)
    while emprestimo_id in emprestimos_ids:
        emprestimo_id = fake.unique.random_number(digits=9)
    emprestimos_ids.add(emprestimo_id)
    data_solicitacao = fake.date_time_between(start_date='-2y', end_date='-6mo')
    tipo_emprestimo = random.choice(['Pessoal', 'Automotivo', 'Imobiliário', 'Consignado'])
    valor_emprestimo = round(random.uniform(1000, 50000), 2)
    taxa_juros = round(random.uniform(0.01, 0.05), 4)
    prazo_meses = random.randint(12, 60)
    data_aprovacao = datetime.now()  # Data de aprovação = timestamp atual
    status = 'Aprovado'  # Novos empréstimos começam como aprovados
    return pd.DataFrame([{
        'emprestimo_id': emprestimo_id,
        'cliente_id': cliente_id,
        'data_solicitacao': data_solicitacao,
        'tipo_emprestimo': tipo_emprestimo,
        'valor_emprestimo': valor_emprestimo,
        'taxa_juros': taxa_juros,
        'prazo_meses': prazo_meses,
        'data_aprovacao': data_aprovacao,
        'status': status
    }])

# --- Função para gerar os pagamentos para um novo empréstimo ---
def generate_new_emprestimo_pagamentos(emprestimo):
    pagamentos_list = []
    emprestimo_id = emprestimo['emprestimo_id'].iloc[0]
    prazo_meses = emprestimo['prazo_meses'].iloc[0]
    valor_emprestimo = emprestimo['valor_emprestimo'].iloc[0]
    data_aprovacao = emprestimo['data_aprovacao'].iloc[0]
    valor_parcela_teorica = round(valor_emprestimo / prazo_meses, 2)

    for i in range(1, prazo_meses + 1):
        pagamento_id = fake.unique.random_number(digits=10)
        while pagamento_id in pagamentos_ids:
            pagamento_id = fake.unique.random_number(digits=10)
        pagamentos_ids.add(pagamento_id)
        data_vencimento = data_aprovacao + timedelta(days=i * 30)
        pagamentos_list.append({
            'pagamento_id': pagamento_id,
            'emprestimo_id': emprestimo_id,
            'numero_parcela': i,
            'data_vencimento': data_vencimento.date(),
            'data_pagamento': None,
            'valor_pago': 0.0,
            'status_pagamento': 'Pendente'
        })
    return pd.DataFrame(pagamentos_list)

# --- Função para gerar um novo pagamento aleatório (mantido para outros empréstimos) ---
def generate_random_pagamento():
    if emprestimos_df.empty:
        return None
    emprestimo = emprestimos_df.sample(n=1).iloc[0]
    emprestimo_id = emprestimo['emprestimo_id']
    prazo_meses = emprestimo['prazo_meses']
    data_aprovacao = emprestimo['data_aprovacao']
    if data_aprovacao is None or prazo_meses <= 0:
        return None

    numero_parcela = random.randint(1, prazo_meses)
    data_vencimento = data_aprovacao + timedelta(days=numero_parcela * 30)
    data_atual_simulacao = datetime.now()
    data_pagamento_simulada = None
    status_pagamento = 'Pendente'
    valor_emprestimo = emprestimo['valor_emprestimo']
    valor_parcela_teorica = round(valor_emprestimo / prazo_meses, 2)
    valor_pago = 0.0

    pagamento_id = fake.unique.random_number(digits=10)
    while pagamento_id in pagamentos_ids:
        pagamento_id = fake.unique.random_number(digits=10)
    pagamentos_ids.add(pagamento_id)

    if data_vencimento < data_atual_simulacao:
        if random.random() < 0.7:
            data_pagamento_simulada = data_vencimento + timedelta(days=random.randint(0, 10))
            valor_pago = round(random.uniform(valor_parcela_teorica * 0.8, valor_parcela_teorica * 1.1), 2)
            status_pagamento = 'Pago'
        else:
            status_pagamento = 'Em Atraso'
    else:
        if random.random() < 0.8:
            data_pagamento_simulada = data_vencimento - timedelta(days=random.randint(0, 5))
            valor_pago = round(random.uniform(valor_parcela_teorica * 0.9, valor_parcela_teorica * 1.05), 2)
            status_pagamento = 'Pago'

    return pd.DataFrame([{
        'pagamento_id': pagamento_id,
        'emprestimo_id': emprestimo_id,
        'numero_parcela': numero_parcela,
        'data_vencimento': data_vencimento.date(),
        'data_pagamento': data_pagamento_simulada.date() if data_pagamento_simulada else None,
        'valor_pago': valor_pago if status_pagamento == 'Pago' else 0.0,
        'status_pagamento': status_pagamento
    }])

# --- Função para escrever o dataframe em uma tabela PostgreSQL ---
def write_dataframe_to_rds(dataframe, table_name, db_host, db_port, db_name, db_user, db_password):
    if dataframe is not None and not dataframe.empty:
        try:
            engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
            dataframe.to_sql(table_name, engine, if_exists='append', index=False)
            print(f"Novo registro adicionado à tabela '{table_name}' no RDS.")
        except Exception as e:
            print(f"Erro ao escrever na tabela '{table_name}' no RDS: {e}")
        finally:
            if 'engine' in locals():
                engine.dispose()

# --- Configurações do Banco de Dados RDS ---
DB_HOST = 'db-banco.c07gis2620tg.us-east-1.rds.amazonaws.com'
DB_PORT = '5432'
DB_NAME = 'db_banco'
DB_USER = 'postgres'
DB_PASSWORD = 'XqNuKeQw12345'

def add_data_realtime():
    global clientes_df
    global transacoes_df
    global emprestimos_df
    global pagamentos_df
    while True:
        time.sleep(5)
        choice = random.choice(['cliente', 'transacao', 'pagamento'])
        if choice == 'cliente':
            new_cliente = generate_new_cliente()
            clientes_df = pd.concat([clientes_df, new_cliente], ignore_index=True)
            write_dataframe_to_rds(new_cliente, 'clientes', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
            # Gera um empréstimo para o novo cliente imediatamente
            if not new_cliente.empty:
                cliente_id = new_cliente['cliente_id'].iloc[0]
                new_emprestimo_df = generate_new_emprestimo(cliente_id)
                emprestimos_df = pd.concat([emprestimos_df, new_emprestimo_df], ignore_index=True)
                write_dataframe_to_rds(new_emprestimo_df, 'emprestimos', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
                # Gera os pagamentos para o novo empréstimo
                new_pagamentos_df = generate_new_emprestimo_pagamentos(new_emprestimo_df)
                pagamentos_df = pd.concat([pagamentos_df, new_pagamentos_df], ignore_index=True)
                write_dataframe_to_rds(new_pagamentos_df, 'pagamentos', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
                # Gera uma transação para o novo cliente
                new_transacao_df = generate_new_transacao()
                if new_transacao_df is not None:
                    transacoes_df = pd.concat([transacoes_df, new_transacao_df], ignore_index=True)
                    write_dataframe_to_rds(new_transacao_df, 'transacoes', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)

        elif choice == 'transacao':
            if not clientes_df.empty:
                new_transacao = generate_new_transacao()
                if new_transacao is not None:
                    transacoes_df = pd.concat([transacoes_df, new_transacao], ignore_index=True)
                    write_dataframe_to_rds(new_transacao, 'transacoes', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
                else:
                    print("Não há clientes para gerar transação.")
        elif choice == 'pagamento':
            if not emprestimos_df.empty:
                new_pagamento = generate_random_pagamento()
                if new_pagamento is not None:
                    pagamentos_df = pd.concat([pagamentos_df, new_pagamento], ignore_index=True)
                    write_dataframe_to_rds(new_pagamento, 'pagamentos', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
                else:
                    print("Não há empréstimos para gerar pagamento.")

if __name__ == "__main__":
    # Geração inicial de alguns clientes e seus respectivos empréstimos e pagamentos
    for _ in range(3):
        new_cliente = generate_new_cliente()
        clientes_df = pd.concat([clientes_df, new_cliente], ignore_index=True)
        write_dataframe_to_rds(new_cliente, 'clientes', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
        # Gera um empréstimo para cada cliente inicial
        if not new_cliente.empty:
            cliente_id = new_cliente['cliente_id'].iloc[0]
            new_emprestimo_df = generate_new_emprestimo(cliente_id)
            emprestimos_df = pd.concat([emprestimos_df, new_emprestimo_df], ignore_index=True)
            write_dataframe_to_rds(new_emprestimo_df, 'emprestimos', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
            # Gera os pagamentos para o novo empréstimo inicial
            new_pagamentos_df = generate_new_emprestimo_pagamentos(new_emprestimo_df)
            pagamentos_df = pd.concat([pagamentos_df, new_pagamentos_df], ignore_index=True)
            write_dataframe_to_rds(new_pagamentos_df, 'pagamentos', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
            # Gera uma transação para o novo cliente inicial
            new_transacao_df = generate_new_transacao()
            if new_transacao_df is not None:
                transacoes_df = pd.concat([transacoes_df, new_transacao_df], ignore_index=True)
                write_dataframe_to_rds(new_transacao_df, 'transacoes', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)

    # Geração inicial de algumas transações e pagamentos aleatórios
    for _ in range(5):
        new_transacao = generate_new_transacao()
        if new_transacao is not None:
            transacoes_df = pd.concat([transacoes_df, new_transacao], ignore_index=True)
            write_dataframe_to_rds(new_transacao, 'transacoes', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)

    for _ in range(5):
        new_pagamento = generate_random_pagamento()
        if new_pagamento is not None:
            pagamentos_df = pd.concat([pagamentos_df, new_pagamento], ignore_index=True)
            write_dataframe_to_rds(new_pagamento, 'pagamentos', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)

    # Inicia a adição de dados em tempo real em uma thread separada
    realtime_thread = threading.Thread(target=add_data_realtime)
    realtime_thread.daemon = True
    realtime_thread.start()

    print("Adicionando novos usuários (e um empréstimo com pagamentos pendentes e uma transação para cada), transações e pagamentos aleatórios a cada 5 segundos...")
    print("Mantenha o script rodando para continuar a adicionar dados.")

    # Mantém o thread principal rodando para que a thread de adição continue
    while True:
        time.sleep(1)

Novo registro adicionado à tabela 'clientes' no RDS.
Novo registro adicionado à tabela 'emprestimos' no RDS.
Novo registro adicionado à tabela 'pagamentos' no RDS.


KeyboardInterrupt: 